In [ ]:
!pip install langchain langchain-community sentence-transformers faiss-cpu llama-cpp-python

from IPython.display import clear_output
clear_output()

import zipfile, os
from google.colab import files

uploaded = files.upload()
zip_filename = list(uploaded.keys())[0]

extract_dir = "/content/md_docs"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

md_files = [os.path.join(root, f)
            for root, _, files in os.walk(extract_dir)
            for f in files if f.endswith('.md')]

selected_file = md_files[0]

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = []
loader = TextLoader(selected_file)
documents.extend(loader.load())

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
docs = splitter.split_documents(documents)

model_path = "/content/tinyllama.gguf"
if not os.path.exists(model_path):
    !wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_0.gguf -O {model_path}

from langchain.llms import LlamaCpp

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.3,
    max_tokens=1024,
    n_ctx=2048,
    top_p=1,
    verbose=True
)

from langchain.chains.summarize import load_summarize_chain

chain = load_summarize_chain(llm, chain_type="stuff")
summary = chain.run(docs)

from rouge_score import rouge_scorer

reference_md_for_rouge = selected_file
with open(reference_md_for_rouge, 'r', encoding='utf-8') as f:
    reference_content = f.read()

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_content, summary)
for key in scores:
    print(f'{key}: {scores[key]}')

print(summary)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 19.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
